# Paralelismo
Se refiere a utilizar varios procesos al mismo tiempo. Con julia se puede hacer en una sola computadora o distribuido. Para aprovechar los nucleos de procesamiento de tu computadora, es conveniente iniciar Julia un número equivalente de `workers` o trabajadores. En mi caso ese número es 4:
```shell
julia -p 4
```
Al igual que una co-rutina, para hacer funcionar a los trabajadores, hay que crear una llamada remota `remotecall()`. Ésta recive la función a processar, y sus argumentos:

In [ ]:
r = remotecall(rand, 2, 2, 2)

Para tomar un valor de ésta función, se debe mandar llamar a función `fetch()` con el trabajador de quien se quiere recolectar la información:

In [ ]:
fetch(r)

**NOTA** los trabajadores no tienen acceso al scope global, por lo que es necesario cargar las lbrerías para cáda worker.

In [ ]:
function rand2(dims...)
    return 2*rand(dims...)
end
rand2(2,2)
fetch(@spawn rand2(2,2))

```shell
julia -p <n> -L módulo1.jl -L módulo2.jl MiPrograma.jl
```

# Simulación de montecarlo
Modelado a partir de eventos aleatorios.

In [ ]:
# en Contar.jl
function contar(n)
    c::Int = 0
    for i=1:n
        c += rand(Bool)
    end
    c
end

In [ ]:
# En programa principal
@everywhere include("Contar.jl")

a = @spawn contar(100000000)
b = @spawn contar(100000000)
fetch(a)+fetch(b) # REDUCCIÓN!

Noten que estámos usando únicamente dos workers, cuando la mayoría de nosotros tiene por lo menos cuatro nucleos. El uso generalizado de workers se puede escribir con loops paralelos:

In [ ]:
n = @parallel (+) for i=1:200000000 # La reducción se lleva a cabo por la función +
  Int(rand(Bool))
end

# Mapeo en paralelo
Hablamos ya de la función `map()` y cómo ésta nos ayuda a mapear una función a un arreglo. Para utilizarla en paralelo, ya que la implementación a bajo nivel es ligeramente distinta, el nombre es `pmap()` (Parallel Maping). Esta función se encarga de organizar a los workers y evita que tengas que realizar llamadas remotas.

In [ ]:
M = Matrix{Float64}[rand(1000,1000) for i=1:10]
pmap(svd, M)

Eviten usar arreglos dentro de un ciclo for paralelizado, pues [normalmente](http://docs.julialang.org/en/release-0.5/manual/parallel-computing.html#shared-arrays) los arreglos no son compartidos entre trabajadores.

In [ ]:
a = SharedArray(Float64,10)
@parallel for i=1:10
  a[i] = i # NOT
end

# Clusters
Julia puede ser iniciado en paralelo con la opción `--machinefile` que lanzará un trabajador por cada linea en el archivo indicado. Las máquinas definidas en el archivo deben estár accesibles por medio de SSH sin contraseña, y tener julia instalado. El archivo de definición de máquinas debe tener los siguientes campos:
```shell
[count*][user@]host[:port] [bind_addr[:port]] 
```
Dónde:
- `user` tiene por default el mismo usuario que el acutal
- `port` es el puerto default de SSH (22)
- `count` es el número de workers a trabajar en el nódo (default 1)
- La opción `bind_addr[:port]` especifica la ip y puerto a la que otros workers deben entrar para conectarse con el actual.

A darle! :)
